In [577]:
import time

import os, sys
import pandas as pd, numpy as np

In [7]:
__scrobbles__ = 'downloads/gps56-2.csv'

In [8]:
pd.options.mode.chained_assignment = None

In [9]:
weekday_map = {0: 'monday',
              1: 'tuesday', 
              2: 'wednesday',
              3: 'thursday', 
              4: 'friday', 
              5: 'saturday', 
              6: 'sunday'}

In [10]:
df = pd.read_csv(__scrobbles__, header = None, names = ['artist', 'album', 'song', 'datetime'])

In [11]:
# remove where datetime is NaN (negligible)
df = df[~df.datetime.isna()]

In [12]:
# convert datetime column to acutal datetime objects
# subtract 7 hours since original times are UTC
df.datetime = pd.to_datetime(df.datetime) - pd.Timedelta(hours = 7)

In [13]:
df.tail(20)

,artist,album,song,datetime
55538,Getter,Wat The Frick EP,Something New,2017-07-05 06:37:00
55539,Che Ecru,buries,2 AM,2017-07-05 06:34:00
55540,Deorro,Good Evening,Guide Me,2017-07-05 06:31:00
55541,BONJR,Hey Sego,hey sego,2017-07-05 06:28:00
55542,Oshi,oshi,state of flux,2017-07-05 06:24:00
55543,Young Thug,Slime Season 3,With Them,2017-07-05 06:20:00
55544,Hyper Potions,Adventures,Adventures,2017-07-04 20:53:00
55545,Young Thug,Beautiful Thugger Girls,Daddy's Birthday,2017-07-04 13:05:00
55546,Young Thug,Slime Season,Again,2017-07-04 13:01:00
55547,Young Thug,Slime Season,Overdosin,2017-07-04 12:50:00


# Time Analysis

In [32]:
df.datetime

1       2020-05-04 17:34:00
2       2020-05-04 15:07:00
3       2020-05-04 15:05:00
4       2020-05-04 15:05:00
5       2020-05-04 14:58:00
                ...        
55553   2017-07-04 12:26:00
55554   2017-07-04 12:23:00
55555   2017-07-01 14:33:00
55556   2017-06-29 20:39:00
55557   2017-06-29 20:34:00
Name: datetime, Length: 55281, dtype: datetime64[ns]

In [42]:
df.datetime.dt.weekday.map(weekday_map).value_counts()

thursday     9829
friday       9606
tuesday      9445
wednesday    7775
monday       7201
saturday     7029
sunday       4396
Name: datetime, dtype: int64

# Lyric Sentiment Analysis